## Copyright 2023 Google LLC

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
import torch

t = torch.tensor([1, 2, 3, 5, 6, 6, 6])
idx = torch.argmax(t, keepdim=True) 
t.scatter_(-1, idx, 2, reduce='multiply')
t

tensor([ 1,  2,  3,  5, 12,  6,  6])

In [15]:
import math
import torch
t = torch.tensor([[1, 2, 4], [1, 2, 5]], dtype=torch.float64)
idx = torch.argmax(t, -1, keepdim=True)
means = t.mean(-1, keepdim=True)
# t.shape, idx.shape, means.shape
t.scatter_reduce_(-1, idx, -means, reduce='sum')

# math.sqrt(0.2) * (value - mean) + mean

tensor([[1.0000, 2.0000, 1.6667],
        [1.0000, 2.0000, 2.3333]], dtype=torch.float64)

In [23]:
cumsum = torch.tensor([[0.4, 0.6, 0.8, 1, 1], [0.4, 0.6, 0.8, 1, 1]], dtype=torch.float64)
s = torch.tensor([[40, 20, 20, 20, 0], [40, 20, 20, 20, 0]], dtype=torch.float64)

s[:, 0] = 0

idx = torch.argmax(s, dim=-1, keepdim=True)
bound = torch.gather(cumsum, dim=-1, index=idx)
# print(idx, bound)

# надо найти prev index, чтобы узнать, как сильно мы обнулили
prev_idx = idx.detach().clone()
prev_idx[prev_idx != 0] -= 1
prev_bound = torch.gather(cumsum, dim=-1, index=prev_idx)
# print(prev_idx, prev_bound)

# это вклад конкретно нужного элемента в дисперсии
diff = (bound - prev_bound) 
remained = 0.5 - prev_bound
# print('diff:', diff)
# print(remained)
diff[remained != 0] = remained[remained != 0] / diff[remained != 0]
# print('other diff:', diff)
diff[remained == 0] = 0
# print('new diff:',diff)
s.scatter_reduce_(-1, idx, diff, reduce='prod') 
s

tensor([[1],
        [1]]) tensor([[0.6000],
        [0.6000]], dtype=torch.float64)
tensor([[0],
        [0]]) tensor([[0.4000],
        [0.4000]], dtype=torch.float64)
diff: tensor([[0.2000],
        [0.2000]], dtype=torch.float64)
tensor([[0.1000],
        [0.1000]], dtype=torch.float64)
other diff: tensor([[0.5000],
        [0.5000]], dtype=torch.float64)
new diff: tensor([[0.5000],
        [0.5000]], dtype=torch.float64)


tensor([[ 0.,  5., 10., 10.,  0.],
        [ 0.,  5., 10., 10.,  0.]], dtype=torch.float64)

In [48]:
import torch
import numpy as np

t = torch.randn((1, 2, 3, 5, 6))
# (t < 3).nonzero().shape

# обнулила тут 
# нахожу индекс
idx = torch.argmax(t, dim=-1, keepdim=True)
idx = torch.max(0, torch.argmin(t, dim=-1, keepdim=True) - 1)
torch.gather(t, -1, idx).shape

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [23]:
torch.argmin(t, dim=-1, keepdim=False)

tensor([[[[5, 4, 5, 3, 0],
          [5, 0, 5, 4, 2],
          [4, 1, 4, 2, 4]],

         [[3, 5, 5, 4, 0],
          [4, 5, 3, 2, 1],
          [3, 3, 5, 1, 4]]]])

In [9]:
t.shape

torch.Size([1, 2, 3])

In [4]:
t

tensor([1, 2, 3, 4])

# StyleAligned over SD1.4

In [ ]:
from diffusers import DDIMScheduler,StableDiffusionPipeline
import torch
import mediapy
import sa_handler
import math

In [ ]:
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False,
                          set_alpha_to_one=False)
pipeline = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16",
    scheduler=scheduler
)
pipeline = pipeline.to("cuda")

handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(share_group_norm=True,
                                      share_layer_norm=True,
                                      share_attention=True,
                                      adain_queries=True,
                                      adain_keys=True,
                                      adain_values=False,
                                     )

handler.register(sa_args, )

In [ ]:
# run StyleAligned

sets_of_prompts = [
  "a toy train. macro photo. 3d game asset",
  "a toy airplane. macro photo. 3d game asset",
  "a toy bicycle. macro photo. 3d game asset",
  "a toy car. macro photo. 3d game asset",
  "a toy boat. macro photo. 3d game asset",
]
images = pipeline(sets_of_prompts, generator=None).images
mediapy.show_images(images)